# Tensors and Operations

In [1]:
import torch

print(torch.__version__)

2.0.0+cpu


In [2]:
# 0 dimensional tensor (scalar)
s = torch.tensor(7)
s.ndim, s.shape

(0, torch.Size([]))

In [5]:
# 1 dimensional tensor (vector)
t1 = torch.Tensor([1,2,3,4,5])
t1.ndim, t1.shape

(1, torch.Size([5]))

In [6]:
# 2 dimensional tensor (matrix)
t2 = torch.Tensor([[1,2,3],
                   [4,5,6],
                   [7,8,9]])

t2.ndim, t2.shape

(2, torch.Size([3, 3]))

In [6]:
# random, zeros and ones
r = torch.rand(size = (3, 3))
print(r)

z = torch.zeros(size = (3, 1))
print(z)

o = torch.ones(size = (1, 3))
print(o)

tensor([[0.1233, 0.2824, 0.6731],
        [0.0130, 0.1551, 0.5286],
        [0.0123, 0.5539, 0.1224]])
tensor([[0.],
        [0.],
        [0.]])
tensor([[1., 1., 1.]])


In [7]:
# Create a tensor
some_tensor = torch.rand(3, 4)

# Find out details about it
print(some_tensor)
print(f"Shape of tensor: {some_tensor.shape}")
print(f"Datatype of tensor: {some_tensor.dtype}")
print(f"Device tensor is stored on: {some_tensor.device}") # will default to CPU

tensor([[0.7574, 0.7046, 0.6467, 0.5331],
        [0.5987, 0.1539, 0.4576, 0.4937],
        [0.2268, 0.8108, 0.4532, 0.3962]])
Shape of tensor: torch.Size([3, 4])
Datatype of tensor: torch.float32
Device tensor is stored on: cpu


In [14]:
mat1 = torch.rand(2,3)
mat2 = torch.rand(3,1)

mat1@mat2 #dot product (same as torch.matmul(mat1, mat2))

tensor([[0.9928],
        [0.6828]])

## Linear

In [17]:
# Shapes need to be in the right way  
tensor_A = torch.tensor([[1, 2],
                         [3, 4],
                         [5, 6]], dtype=torch.float32)

tensor_B = torch.tensor([[7, 10],
                         [8, 11], 
                         [9, 12]], dtype=torch.float32)

torch.matmul(tensor_A, tensor_B.T) # (this will error)

tensor([[ 27.,  30.,  33.],
        [ 61.,  68.,  75.],
        [ 95., 106., 117.]])

In [26]:
# Since the linear layer starts with a random weights matrix, let's make it reproducible (more on this later)
torch.manual_seed(42)
# This uses matrix multiplication
linear = torch.nn.Linear(in_features=2,  # in_features = matches inner dimension of input 
                         out_features=6,
                         bias=True) # out_features = describes outer value 
x = tensor_A
output = linear(x)
print(f"Input shape: {x.shape}\n")
print(f"Output:\n{output}\n\nOutput shape: {output.shape}")

Input shape: torch.Size([3, 2])

Output:
tensor([[2.2368, 1.2292, 0.4714, 0.3864, 0.1309, 0.9838],
        [4.4919, 2.1970, 0.4469, 0.5285, 0.3401, 2.4777],
        [6.7469, 3.1648, 0.4224, 0.6705, 0.5493, 3.9716]],
       grad_fn=<AddmmBackward0>)

Output shape: torch.Size([3, 6])


In [28]:
linear.weight, linear.bias

(Parameter containing:
 tensor([[ 0.5406,  0.5869],
         [-0.1657,  0.6496],
         [-0.1549,  0.1427],
         [-0.3443,  0.4153],
         [ 0.6233, -0.5188],
         [ 0.6146,  0.1323]], requires_grad=True),
 Parameter containing:
 tensor([ 0.5224,  0.0958,  0.3410, -0.0998,  0.5451,  0.1045],
        requires_grad=True))

## Min, Max, ArgMin, ArgMax.

Basically the same as numpy

In [32]:
# Create a tensor
x = torch.arange(0, 100, 10).reshape(2,5)
x

tensor([[ 0, 10, 20, 30, 40],
        [50, 60, 70, 80, 90]])

In [36]:
# require casting to float for mean
torch.max(x), torch.min(x), torch.mean(x.type(torch.float32)), torch.sum(x)

(tensor(90), tensor(0), tensor(45.), tensor(450))

In [49]:
a = x.reshape(1, -1).squeeze()
a

tensor([ 0, 10, 20, 30, 40, 50, 60, 70, 80, 90])

In [58]:
torch.cuda.is_available()

False